In [ ]:
import os
import time
import csv

# Import shared loader
from Shared_Components import load_tasks_from_csv

# Import the 3 algorithms directly
from Greedy import run_greedy
from Ant_Colony import run_ant_colony
from Dynamic_Scheduling import run_dynamic


# ------------------------------------------------------------
# Time a function
# ------------------------------------------------------------
def measure(func, tasks):
    start = time.time()
    result = func(tasks)
    return time.time() - start, result


# ------------------------------------------------------------
# Write optimized schedule CSV
# With new first column = ORIGINAL CSV INDEX
# ------------------------------------------------------------
def write_schedule_csv(output_name, original_tasks, optimized_list):
    """
    original_tasks: list of Task objects from original CSV
    optimized_list: list of Task objects in final schedule order
    """

    # Map task name → original index in CSV
    index_map = {}
    for i, t in enumerate(original_tasks):
        index_map[t.name] = i

    with open(output_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["original_index", "start", "end", "weight"])

        for t in optimized_list:
            writer.writerow([
                index_map[t.name],   # new first column
                t.start,
                t.end,
                t.priority
            ])


# ------------------------------------------------------------
# Pretty-print schedule with newlines every 3 tasks
# ------------------------------------------------------------
def print_schedule(title, schedule):
    print(f"\n--- {title} Schedule (formatted) ---")
    for i, t in enumerate(schedule):
        print(t, end="  ")
        if (i + 1) % 3 == 0:
            print()
    print("\n")


# ------------------------------------------------------------
# Main execution: loop through ALL CSV files
# ------------------------------------------------------------
if __name__ == "__main__":
    folder = input("Enter folder containing CSV schedule files: ").strip()

    folder = os.path.expanduser(folder)
    folder = os.path.abspath(folder)

    print(f"Using folder: {folder}")

    if not os.path.isdir(folder):
        print("Error: Folder not found.")
        exit(1)

    results = {}
    first_file_processed = False

    csv_files = [f for f in os.listdir(folder) if f.lower().endswith(".csv")]
    csv_files.sort()

    for filename in csv_files:
        full_path = os.path.join(folder, filename)
        print(f"\n=== Processing {filename} ===")

        tasks = load_tasks_from_csv(full_path)
        print(f"Loaded {len(tasks)} tasks")

        # ---- time algorithms AND retrieve their schedules ----
        t_g, out_g = measure(run_greedy, tasks)
        t_a, out_a = measure(run_ant_colony, tasks)
        t_d, out_d = measure(run_dynamic, tasks)

        # Save times
        results[filename] = {
            "greedy": t_g,
            "aco": t_a,
            "dynamic": t_d
        }

        print(f"Greedy:  {t_g:.6f}s")
        print(f"ACO:     {t_a:.6f}s")
        print(f"Dynamic: {t_d:.6f}s")

        # ---- For the FIRST CSV file only: output 3 optimized CSVs ----
        if not first_file_processed:
            print("\nCreating optimized schedule CSVs for FIRST file only...")

            write_schedule_csv("optimized_greedy.csv", tasks, out_g)
            write_schedule_csv("optimized_aco.csv", tasks, out_a)
            write_schedule_csv("optimized_dynamic.csv", tasks, out_d)

            # Print schedules nicely
            print_schedule("Greedy", out_g)
            print_schedule("ACO", out_a)
            print_schedule("Dynamic", out_d)

            first_file_processed = True

    # --------------------------------------------------------
    # Write algorithm timing CSV (3 columns, 1 row per CSV)
    # --------------------------------------------------------
    with open("algorithm_timings.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "greedy", "aco", "dynamic"])

        for fname, data in results.items():
            writer.writerow([
                fname,
                f"{data['greedy']:.6f}",
                f"{data['aco']:.6f}",
                f"{data['dynamic']:.6f}"
            ])

    print("\nTiming results written to algorithm_timings.csv")
    print("Optimized schedules written for the FIRST CSV only:")
    print("   optimized_greedy.csv")
    print("   optimized_aco.csv")
    print("   optimized_dynamic.csv")

In [ ]:
#Date: 11/11/2025
#Team Members: Remek Botwright, Soren Richards, Danny English, Luke Sapp
#Professor: Cristina Radian
#------------------------------------------------------------
#
#Overview:
#This project explores telescope scheduling — a process used to efficiently
#allocate limited resources (like telescope time) to high-priority tasks.
#We’ll implement and compare three algorithms: Ant Colony, Greedy, and Dynamic
#Scheduling. The goal is to evaluate efficiency, adaptability, and performance
#across each method.
#
#------------------------------------------------------------
#"""

# ------------------------------------
# Shared Data Structures + CSV Loader
# ------------------------------------

import csv
import random
import math
import time

class Task:
    def __init__(self, name, start_time, end_time, priority):
        self.name = name
        self.start = float(start_time)
        self.end = float(end_time)
        self.priority = int(priority)

    def __repr__(self):
        return f"{self.name}(Priority={self.priority}, {self.start}-{self.end})"

def load_tasks_from_csv(path):
    tasks = []

    with open(path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)

        # Validate required columns
        required = {"start", "end", "weight"}
        if not required.issubset(reader.fieldnames):
            raise ValueError(
                f"CSV {path} must contain the columns: {', '.join(required)}"
            )

        for i, row in enumerate(reader):
            name = f"Task{i+1}"

            start = float(row["start"])
            end = float(row["end"])

            # Convert weight → priority (still integer 1–9)
            priority = int(row["weight"])

            tasks.append(Task(name, start, end, priority))

    return tasks


In [ ]:
def greedy_schedule(tasks):
    # Sort by: 1) highest weight (priority), 2) earliest end time
    tasks_sorted = sorted(tasks, key=lambda t: (-t.priority, t.end))

    schedule = []
    current_end = -1

    for task in tasks_sorted:
        if task.start >= current_end:
            schedule.append(task)
            current_end = task.end

    return schedule

def run_greedy(tasks):
    print("\n🪙 Running Greedy Scheduler...")
    result = greedy_schedule(tasks)
    print("Greedy Schedule:", result)
    return result

In [ ]:
from Shared_Components import *

class DynamicScheduler:
    def __init__(self):
        self.active_tasks = []
        self.schedule = []
        self.current_time = 0

    def add_task(self, task):
        print(f"[Time {self.current_time}] Adding task {task.name}")
        self.active_tasks.append(task)
        self.reschedule()

    def remove_task(self, task_name):
        print(f"[Time {self.current_time}] Removing task {task_name}")
        self.active_tasks = [t for t in self.active_tasks if t.name != task_name]
        self.reschedule()

    def reschedule(self):
        # Sort by weight, then earliest finish time
        tasks_sorted = sorted(self.active_tasks,
                              key=lambda t: (-t.priority, t.end))

        new_schedule = []
        current_end = -1

        for task in tasks_sorted:
            if task.start >= current_end:
                new_schedule.append(task)
                current_end = task.end

        self.schedule = new_schedule
        print(f"[Time {self.current_time}] New Schedule: {new_schedule}\n")

    def simulate(self, steps=3, delay=0):
        for _ in range(steps):
            self.current_time += 1
            if delay:
                time.sleep(delay)
            print(f"Tick {self.current_time}: Active schedule: {self.schedule}")

def run_dynamic(tasks):
    print("\n🔁 Running Dynamic Scheduler...")
    scheduler = DynamicScheduler()
    for task in tasks:
        scheduler.add_task(task)

    scheduler.simulate(2)
    return scheduler.schedule

In [ ]:
#"""
#------------------------------------------------------------
#Project: Telescopic Scheduling
#Course: CSC 2400-002
#Date: 11/11/2025
#Team Members: Remek Botwright, Soren Richards, Danny English, Luke Sapp
#Professor: Cristina Radian
#------------------------------------------------------------
#
#Overview:
#This project explores telescope scheduling — a process used to efficiently
#allocate limited resources (like telescope time) to high-priority tasks.
#We’ll implement and compare three algorithms: Ant Colony, Greedy, and Dynamic
#Scheduling. The goal is to evaluate efficiency, adaptability, and performance
#across each method.
#
#------------------------------------------------------------
#"""

# ------------------------------------
# Ant Colony Optimization
# ------------------------------------

from Shared_Components import *
import random
import math

SAFE_EPSILON = 0.000001

class AntColony:
    def __init__(self, tasks, n_ants=8, n_iterations=18,
                 decay=0.1, alpha=1, beta=2):

        self.tasks = tasks
        self.n = len(tasks)

        self.distances = self._build_distance_matrix()
        self.pheromone = [[1.0 for _ in range(self.n)] for _ in range(self.n)]

        self.n_ants = n_ants
        self.n_iterations = n_iterations

        self.decay = decay      # pheromone evaporation
        self.alpha = alpha      # pheromone importance
        self.beta = beta        # heuristic importance

    # -----------------------------------------------------------
    # Distance Matrix
    # - overlap gives huge penalty
    # - time gap is small penalty
    # - NO negative distances (bad for heuristic)
    #
    # interpretation: LOWER = better transition
    # -----------------------------------------------------------
    def _build_distance_matrix(self):
        n = len(self.tasks)
        M = [[0]*n for _ in range(n)]

        for i in range(n):
            A = self.tasks[i]

            for j in range(n):
                if i == j:
                    M[i][j] = float("inf")
                    continue

                B = self.tasks[j]

                # Overlap penalty (huge)
                overlap = max(0, min(A.end, B.end) - max(A.start, B.start))
                if overlap > 0:
                    M[i][j] = 1e8  # don't allow transitions into overlaps
                    continue

                # Legit transition → small time gap penalty
                gap = max(0, B.start - A.end)

                # Weight helps reduce distance (slightly)
                avg_weight = (A.priority + B.priority) / 2

                dist = gap + (1 / (avg_weight + 1))

                if dist <= 0:
                    dist = SAFE_EPSILON

                M[i][j] = dist

        return M

    # -----------------------------------------------------------
    # Run ACO
    # -----------------------------------------------------------
    def run(self):
        best_weight = -1
        best_path = None

        for iteration in range(self.n_iterations):
            paths = self._construct_all_paths()
            self._evaporate_pheromones()
            self._reinforce_pheromones(paths)

            for path_weight, path in paths:
                if path_weight > best_weight:
                    best_weight = path_weight
                    best_path = path

        final_schedule = self._build_final_schedule(best_path)
        return final_schedule, best_weight

    # -----------------------------------------------------------
    # Build paths for all ants
    # -----------------------------------------------------------
    def _construct_all_paths(self):
        results = []
        for _ in range(self.n_ants):
            path = self._generate_path()
            weight = sum(self.tasks[i].priority for i in path)
            results.append((weight, path))
        return results

    # -----------------------------------------------------------
    # Generate one ant's path
    # -----------------------------------------------------------
    def _generate_path(self):
        start = random.randint(0, self.n - 1)
        used = {start}
        path = [start]

        while True:
            nxt = self._select_next(path[-1], used)
            if nxt is None:
                break
            used.add(nxt)
            path.append(nxt)

        return path

    # -----------------------------------------------------------
    # Select next job via probability:
    #   (pheromone^α) * (heuristic^β)
    #
    # heuristic = (priority / (distance + duration))
    #
    # -----------------------------------------------------------
    def _select_next(self, current, used):
        candidates = []

        for j in range(self.n):
            if j in used:
                continue

            dist = self.distances[current][j]
            if dist >= 1e7:   # impossible overlap transition
                continue

            B = self.tasks[j]
            duration = max(B.end - B.start, SAFE_EPSILON)

            pher = self.pheromone[current][j] ** self.alpha
            heuristic = (B.priority / duration) / dist

            if heuristic <= 0:
                heuristic = SAFE_EPSILON

            score = pher * (heuristic ** self.beta)
            candidates.append((j, score))

        if not candidates:
            return None

        # Roulette Wheel Selection
        total = sum(s for _, s in candidates)
        r = random.random() * total
        running = 0

        for j, score in candidates:
            running += score
            if running >= r:
                return j

        return candidates[-1][0]

    # -----------------------------------------------------------
    # Pheromone Update
    # Ants with HIGH TOTAL WEIGHT deposit more pheromone
    # -----------------------------------------------------------
    def _reinforce_pheromones(self, paths):
        for weight, path in paths:
            if weight <= 0:
                continue

            deposit = weight ** 1.3  # reward exponential growth

            for i in range(len(path) - 1):
                a = path[i]
                b = path[i+1]
                self.pheromone[a][b] += deposit

    # -----------------------------------------------------------
    def _evaporate_pheromones(self):
        for i in range(self.n):
            for j in range(self.n):
                self.pheromone[i][j] *= (1 - self.decay)
                if self.pheromone[i][j] < SAFE_EPSILON:
                    self.pheromone[i][j] = SAFE_EPSILON

    # -----------------------------------------------------------
    # Build final non-overlapping schedule from the best path
    # -----------------------------------------------------------
    def _build_final_schedule(self, path):
        tasks_sorted = sorted([self.tasks[i] for i in path],
                              key=lambda t: t.start)

        schedule = []
        current_end = -1

        for t in tasks_sorted:
            if t.start >= current_end:
                schedule.append(t)
                current_end = t.end

        return schedule


def run_ant_colony(tasks):
    print("\n🐜 Running Improved Ant Colony Optimization...")
    colony = AntColony(tasks)
    schedule, score = colony.run()
    print("ACO Schedule Weight:", score)
    print("ACO Count:", len(schedule))
    return schedule


In [ ]:
#"""
#------------------------------------------------------------
#Project: Telescopic Scheduling
#Course: CSC 2400-002
#Date: 11/11/2025
#Team Members: Remek Botwright, Soren Richards, Danny English, Luke Sapp
#Professor: Cristina Radian
#------------------------------------------------------------
#
#Overview:
#This project explores telescope scheduling — a process used to efficiently
#allocate limited resources (like telescope time) to high-priority tasks.
#We’ll implement and compare three algorithms: Ant Colony, Greedy, and Dynamic
#Scheduling. The goal is to evaluate efficiency, adaptability, and performance
#across each method.
#
#------------------------------------------------------------
#"""

# Dynamic Scheduling

from Shared_Components import load_tasks_from_csv
from Dynamic_Scheduling import run_dynamic

if __name__ == "__main__":
    filename = input("Enter path to your tasks CSV file (e.g., tasks.csv): ")
    tasks = load_tasks_from_csv(filename)
    print(f"\nLoaded {len(tasks)} tasks: {tasks}")

    result = run_dynamic(tasks)
    print("\n🔁 Final Dynamic Schedule:", result)
